In [16]:
import rasterio
import numpy as np
from PIL import Image
from pathlib import Path
from tqdm import tqdm

In [21]:
input_dir = Path("../local_data/SeaIce_Results_RGB")
output_dir = Path("../local_data/SeaIce_Results_RGB_CONV")
output_dir.mkdir(exist_ok=True)

# Define this once outside the loop
def normalize(band):
    b_min, b_max = band.min(), band.max()
    if b_max == b_min:
        return np.zeros(band.shape, dtype=np.uint8)
    # Perform min-max scaling to 0-255 range
    rescaled = (255 * (band - b_min) / (b_max - b_min))
    return rescaled.astype(np.uint8)

In [22]:
# Get list of files first so tqdm knows the total count
tif_files = list(input_dir.glob("*.tif"))

for tif in tqdm(tif_files, desc="Converting GeoTIFFs"):
    with rasterio.open(tif) as src:
        # read(masked=True) handles specialized NoData values automatically
        data = src.read(masked=True) 
        
        if data.mask.all():
            continue

        if data.shape[0] == 1:
            # Grayscale processing
            img_array = normalize(data[0])
        else:
            # RGB processing: Normalize each band (R, G, B) individually
            # This prevents one bright channel from "washing out" the others
            bands = [normalize(data[i]) for i in range(min(3, data.shape[0]))]
            img_array = np.stack(bands, axis=-1)

        # Save the final result
        Image.fromarray(img_array).save(output_dir / f"{tif.stem}.png")

Converting GeoTIFFs: 100%|██████████| 42/42 [00:11<00:00,  3.81it/s]


In [24]:
# get list of them for latex later

# 1. Setup paths
input_dir = Path("../local_data/SeaIce_Results_RGB_CONV")
output_tex = Path("imagelist.tex")

# 2. Find files and strip "Visual_" and ".png"
# This creates a sorted list of IDs
ids = sorted([
    f.stem.replace("RGB_", "") 
    for f in input_dir.glob("RGB_*.png")
])

# 3. Format as a LaTeX command
# This writes: \newcommand{\MyImageList}{id1, id2, id3, ...}
latex_content = "\\newcommand{\\MyImageList}{" + ",\n".join(ids) + "}"

# 4. Save to file
with open(output_tex, "w") as f:
    f.write(latex_content)

print(f"Success! Found {len(ids)} images. Created {output_tex}")

Success! Found 42 images. Created imagelist.tex
